In [1]:
import torch
from absl import logging
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.optimize import curve_fit

import plotly.io as pio
pio.renderers.default = "notebook_connected"
import plotly.graph_objects as go

from icl.linear.train_linear import train
from icl.linear.lr_config import get_config
from icl.linear.lr_task import *
from icl.linear.linear_utils import *
from icl.linear.task_vecs import *
from icl.linear.train_linear import get_sharded_batch_sampler
from icl.linear import DiscreteMMSE, Ridge
from icl.linear.lr_models import MixedRidge, UnbalancedMMSE
#from icl.linear.sufficient_stats import get_sufficient_statistics_fit, get_sufficient_statistics_proj_fit, get_betahat_fit
from icl.utils.linear_notebook_helpers import process_sufficient_statistics, plot_r2_curves_plotly, get_eval_task, process_beta_fit, plot_metrics
from icl.utils.linear_visualization_utils import plot_mse
from icl.utils.linear_ood_analysis import predict_with_ood_vector, enhance_ood, process_ood_evolve
from icl.utils import visualize_attention
from icl.figures.task_vec_viz import *
from icl.utils.experiment_analysis import process_exp
from icl.utils.linear_processor import process_ood_evolve_lambda_metrics

logging.set_verbosity(logging.INFO)
torch.set_printoptions(precision=3, sci_mode=False)
np.set_printoptions(precision=3, suppress=True)

%load_ext autoreload
%autoreload 2

To modify experiment configurations, using the `../src/icl/linear/lr_config.py` file.

In [3]:
config = get_config()
config.task.p_minor = 0.1
config.task.n_minor_tasks = 2**14
config.training.warmup_steps = 1_000
config.training.total_steps = 2_000
model, log = train(config)

..\results\linear\train_7b50641a5f0e9939eb60cee5fcae3849
dict_keys(['train/step', 'train/lr', 'eval/Latent_false', 'eval/Pretrain_false', 'eval/Latent_true', 'eval/Pretrain_true'])


wandb: Currently logged in as: hyan84 (hyan84-university-of-wisconsin-madison) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Start training...
Training complete.


In [5]:
exp_name = "train_7b50641a5f0e9939eb60cee5fcae3849"

# plot sufficient statistics fit

fig, r2_dict = plot_r2_curves_plotly(
    process_beta_fit, exp_name=exp_name,
    layer_indices=range(0, 16), is_eval=True, K=1024
)
fig.show()

C:\Users\User\LLM\mini-ICL\src\icl\linear\linear_utils.py:498: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



In [6]:
process_ood_evolve(exp_name, K=100, layer_index=12, include_minor=True)

Preprocessing...


C:\Users\User\LLM\mini-ICL\src\icl\linear\linear_utils.py:498: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



Too many minority tasks (16384). Randomly sampling 64.
